<a href="https://colab.research.google.com/github/piyush123/CKAD-exercises/blob/master/geo_lookup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- This notebook is meant to be run in Google Colab. -->

# Deep Atlas Primer: Data Spatialization

<small>©️ 2024 Deep Atlas, Inc.</small>


## Exercise: Geo-Lookup


_Searching for vectors in space._


`#vectors` `#faiss` `#dimensions` `#embeddings` `#euclidean-distance` `#l1-distance`


> Objectives:
>
> - Explain how vectors model data in multi-dimensional spaces
> - Use a vector database to perform a similarity search


### Context and goals


Vectors represent a datapoint's location in space:

- Each value or "feature" represents the point's location along a particular dimension.
- The entire vector represents the coordinates of the point in a N-dimensional feature-space.

```python
# Example of a vector: Geographic location in coordinate form:
latitude_longitude_vector = [40.725311738171650, -73.99256233017770]
```

Representing data in vectors unlocks the ability to perform "similarity" searches, based on the distance between datapoints. Points that are closer to each other are considered more similar.

**Goal**: To illustrate vector similarity search, we will set up a database to store location information about restaurants in New York City. We will then query the database to search for restaurants closest to a given geographic point.

- [ ] If you are viewing this on Google Colab and have not already done so, make a copy of this notebook to save your changes (File > Save a copy in Drive).
  - Do not share this notebook, your copy, or any course content without permission.


### Imports


In this exercise, we will store vectors in a _vector store_, a purpose built database that is capable of storing floating-point values and performing distance calculations.

- [ ] Install the FAISS vector store (Facebook AI Similarity Search), a lightweight in-memory vector store:


In [ ]:
%pip install faiss-cpu

We will also be using the Numpy library to create vectors. Numpy allows us to create specialized arrays — with lower-level C/C++ code — to improve the speed and precision of the vector math. This type of array is required by FAISS.

- [ ] Import the FAISS and Numpy modules:


In [ ]:
import numpy as np
import faiss

### Vectorized data


The `restaurants` tuple below defines a list of 15 restaurants from [Eater Manhattan's Hottest Restaurants](https://web.archive.org/web/20231127210302/https://ny.eater.com/maps/best-new-nyc-restaurants-heatmap) (November 2023). You can also see them in [this Google Map](https://maps.app.goo.gl/Tx6kxDwVpmGduKWR6). Both lists are arranged from south-to-north.


In [ ]:
restaurants = [
    {
        "id": "15",
        "name": "Sushi Oku",
        "latlong": [40.715877339351415, -73.99144037455102],
    },
    {
        "id": "14",
        "name": "Foul Witch",
        "latlong": [40.723513837336800, -73.98579742245305],
    },
    {
        "id": "13",
        "name": "Torrisi",
        "latlong": [40.724774782488180, -73.99550349541292],
    },
    {
        "id": "12",
        "name": "Raf's",
        "latlong": [40.725311738171650, -73.99256233017770],
    },
    {
        "id": "11",
        "name": "Superiority Burger",
        "latlong": [40.727675266216245, -73.98319156377961],
    },
    {
        "id": "10",
        "name": "Roscioli",
        "latlong": [40.728588790358934, -74.00245920108593],
    },
    {
        "id": "09",
        "name": "Jazba",
        "latlong": [40.732387507885710, -73.98583346196472],
    },
    {
        "id": "08",
        "name": "Libertine",
        "latlong": [40.733910718772210, -74.00690370630402],
    },
    {
        "id": "07",
        "name": "L'industrie Pizzeria",
        "latlong": [40.734456887168804, -74.00448945047113],
    },
    {
        "id": "06",
        "name": "Cecchi's",
        "latlong": [40.737637412633894, -73.99753672526053],
    },
    {
        "id": "05",
        "name": "Bangkok Supper Club",
        "latlong": [40.739951944575424, -74.00590599736900],
    },
    {
        "id": "04",
        "name": "Cafe Chelsea",
        "latlong": [40.744498683392386, -73.99670547790987],
    },
    {
        "id": "03",
        "name": "Nasrin's Kitchen",
        "latlong": [40.764131773621330, -73.97592055092590],
    },
    {
        "id": "02",
        "name": "Hyderabadi Zaiqa",
        "latlong": [40.764529868414660, -73.98767257976188],
    },
    {
        "id": "01",
        "name": "Tatiana",
        "latlong": [40.772940574906710, -73.98302353372965],
    },
]

- [ ] Use Numpy to create a typed array containing `float32` versions of the latitudes and longitudes of the restaurants.


In [ ]:
# Convert latlong lists into vectors
latlong_vectors = np.array(
    [restaurant["latlong"] for restaurant in restaurants]
).astype("float32")

### Initializing the Vector Store


Next, lets create a vector store instance (also called an index).

To do this with FAISS, we need to specify 2 things:

1. The **number of dimensions** in the vectors
   - For this exercise, we will be dealing with _2-dimensional vectors_ (latitude and longitude).
2. The type of **distance-metric** to use calculate the distance between two points

   - We will use the simplest straight-line distance measure: Euclidean distance (a.k.a. L2 distance)

   <details closed>
   <summary>Learn more about L2 distance (optional)...</summary>

   > The simplest distance measure is _"Euclidean distance"_, also called an _"L2 distance"_ in some libraries. It is the straight-line distance between two points in a Euclidean space (which is to say, a "flat" space without curves in the space itself). We can calculate it using a generalized version of the Pythagorean equation: compute the square-root of the sum of the squares of differences between two vectors' corresponding members.
   >
   > ```python
   > # in 2D, where 0 represents the origin of the space
   > hypotenuse_length = math.sqrt(
   >   (side_1_length - 0) ** 2 + # 0 is the origin
   >   (side_2_length - 0) ** 2
   > )
   >
   > # in N-dimensions
   > euclidean_distance = math.sqrt(
   >   (vector_1[0] - vector_2[0]) ** 2 +
   >   (vector_1[1] - vector_2[1]) ** 2 +
   >   (vector_1[2] - vector_2[2]) ** 2 +
   >   # ... up to (vector_1[N] - vector_2[N]) ** 2
   > )
   > ```

   </details>

- [ ] Initialize a FAISS index using the `IndexFlatL2()` constructor and pass it the number of dimensions:


In [ ]:
# L2 distance (Euclidean distance) index for 2D vectors
index = faiss.IndexFlatL2(2)

- [ ] Add the lat-long vectors to the index:


In [ ]:
index.add(latlong_vectors)
print(f"Number of vectors in the index: {index.ntotal}")

### Querying the vector store


Now that we have our vectors loaded into the index, we can search for data points that are closest — i.e. most similar — to the query vector.

- [ ] Create and format a query vector (a latlong location):


In [ ]:
# Using the latlong for Tatiana:
query_latlong = [40.77294057490671, -73.98302353372965]

query_vector = np.array([query_latlong]).astype("float32")

- [ ] Perform a search for the 5 most similar points:
  - FAISS returns the distances and indices of the closest points. We can use this to retrieve the names of the resturants from the original `restaurants` collection.


In [ ]:
num_neighbors = 5
distances, indices = index.search(query_vector, num_neighbors)

# Display results
print("Indices of nearest neighbors:", indices)
print("Distances to nearest neighbors:", distances)

- [ ] Retrieve the names of the closest items and print the restaurant name and its distance:
  - The `get_neighbors_from_collection` will use the indices and distances returned by FAISS to get the names of the corresponding restaurants.


In [ ]:
def get_neighbors_from_collection(
    collection, attribute_name, indices, distances
):
    nearest_neighbors = [
        (collection[i][attribute_name], distances[0][j])
        for j, i in enumerate(indices[0])
    ]
    return nearest_neighbors


# Get the names of the nearest restaurants:
nearest_neighbors = get_neighbors_from_collection(
    restaurants, "name", indices, distances
)

for name, distance in nearest_neighbors:
    print(f"Restaurant: {name}\n(distance: {distance})\n\n")

You should see Tatiana (with a distance of 0 because we used its coordinates to do the search), followed by Hyderabadi Zaiqa, Nasrin's Kitchen, Cafe Chelsea, and Cecchi's.


### You Did It!


Congratulations — you just performed similarity search! In this exercise we used numeric coordinates in our physical space. The next step will be to transform other forms of data — even text — into vectors representing points in other meaningful spaces, allowing us to go from similarity search to _semantic_ search.

🎉


### Extra Credit: A Taste of ML Debugging


Have you noticed any results that don't make complete sense? (e.g. Bangkok Supper Club not appearing in the results before Cecchi's)

This simple scenario is a perfect example of the problems you can expect to encounter in a typical ML project. When something doesn't work perfectly, there is often no straightforward path to finding the bug. If you're lucky, the symptom might be a surprise result that you can tell is clearly wrong. Or, the only symptom of the bug may be sub-optimal model performance. Worse, you may not even have a benchmark to tell you what the optimal performance could have been!

If you investigate the solution above and the results it provides, you'll find that the relative distances between locations are consistently a little wrong, compared to their crow flight distance.

- [ ] Before reading on, take a moment to try and think of any reasons that could be leading to undesirable results.

Let's find and fix the bug.

If you haven't taken a moment yet to reflect on what might be causing it, please do so before reading on.

It isn't crucial for you to figure out the problem before moving on, but it is crucial that you've made some attempts.


#### Finding the Bug


We're going to spoil the mystery for you. One cause of distortion in our example is that lines of latitude and longitude do not have a reliable translation into distance. The technical way to say this is that we are trying to treat lat/long coordinates as if they lived on a euclidean (that is, a flat and undistorted) space. But the surface of the earth is a 2d space wrapped around a 3d sphere. The resulting distortion is what qualifies the space as a "non-euclidean" space.

So, while a single degree of latitude is approximately the same length throughout the world (~69 miles long), a single degree of longitude varies in length depending on the latitude from which measure the length of that longitudinal degree. At the equator, one degree of longitude is about 69 miles long, and that length shrinks as you approach either pole. In Manhattan, longitudinal degrees are about 52 miles long. Distances calculated by our vector database, however, assume that the units of measurement are equal for both dimensions of the vector. The euclidean distance measurement under-counts longitudinal miles compared to latitudinal miles for locations at any distance from the equator.

This sort of skewing can be solved with "feature engineering" — the mathematical adjustment and editing of features to better suit the task. One example is [converting latitude and longitude values into points in 3-dimensional space](https://math.stackexchange.com/a/29162).


### Extra Credit


- [ ] Implement a solution that counteracts the distortion effects of attempting to do vector similarity search in a curved ("non-euclidean") space.

In this exercise, we set up a vector database, initialized it with data that represented literal points in a non-euclidean 2d space, and used a distance/similarity search to retrieve relevant results. This exercise only explored applications of Euclidean distance measures.

- [ ] Read this post by the purpose-built vector database Pinecone, regarding [vector similarity](https://www.pinecone.io/learn/vector-similarity/), including two other distance measures, cosine (a.k.a. inner product) and dot-product
